In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys

### 주의사항: 슬랙에 공유한 랠릿과 프로그래머스 파일을 현재 폴더에 지정해주세요

In [ ]:
# 파일 불러오기
rallit = pd.read_csv('./rallit_1.csv')  # 랠릿
programmers = pd.read_csv('./programmers.csv')  # 프로그래머스

pro_corp_name = programmers['기업명']  # 프로그래머스 기업명
ral_corp_name = rallit['기업명']  # 랠릿 기업명

### 다트 크롤링
- 크롤링할 컬럼: 기업명, 주소, 업종분류, 설립연도
- 절차
1. 다트 공시현황 사이트 접속
2. 기업명 타이핑하여 공시정보 접속
3. 내용 크롤링
4. 타이핑 되어있는 기업명 제거
5. concat
6. 다음 기업명 타이핑하여 공시정보 접속

위 절차를 반복합니다.

#### 프로그래머스

In [ ]:
# 빈 프레임 생성
pro_extra_res = pd.DataFrame(columns=['기업명','주소','업종분류','설립연도','기업홈페이지URL'])

In [ ]:
# 크롤링 프로그램
url = 'https://dart.fss.or.kr/dsae001/main.do#none'

driver= webdriver.Chrome()
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

for i in pro_corp_name:
    try:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').send_keys(i)  # 키워드 작성
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        time.sleep(.1)
        driver.find_element(By.XPATH, '//*[@id="corpTabel"]/tbody/tr/td[1]/span/a').click()

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = i
        # 주소
        address = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[9]/td').text
        # 업종명
        sector = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[14]/td').text
        # 설립일
        corp_year = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[15]/td').text
        # 기업 홈페이지URL
        corp_url = driver.find_element(By.XPATH, '//*[@id="homePage"]').text

        pro_extra_temp = pd.DataFrame([corp_name,address,sector,corp_year,corp_url], index=['기업명','주소','업종분류','설립연도','기업홈페이지URL']).T
        pro_extra_res = pd.concat([pro_extra_res, pro_extra_temp])

        # 키워드 지우기
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()
        time.sleep(.1)
        
    except:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

driver.quit()

In [ ]:
# 현재 폴더에csv파일로 저장
pro_extra_res.to_csv('./프로그래머스_다트_추가.csv')

#### 랠릿

In [ ]:
# 빈 프레임 생성
ral_extra_res = pd.DataFrame(columns=['기업명','주소','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
url = 'https://dart.fss.or.kr/dsae001/main.do#none'

driver= webdriver.Chrome()
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()  # 키워드가 있을 수 있으므로 지우기

for i in ral_corp_name:  # 기업명 하나씩 불러오기
    try:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').send_keys(i)  # 기업명 작성
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        time.sleep(.1)
        driver.find_element(By.XPATH, '//*[@id="corpTabel"]/tbody/tr/td[1]/span/a').click()

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = i
        # 주소
        address = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[9]/td').text
        # 업종명
        sector = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[14]/td').text
        # 설립일
        corp_year = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[15]/td').text

        ral_extra_temp = pd.DataFrame([corp_name,address,sector,corp_year], index=['기업명','주소','업종분류','설립연도']).T
        ral_extra_res = pd.concat([ral_extra_res, ral_extra_temp])

        # 키워드 지우기
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()
        time.sleep(.1)
        
    except:  # 검색결과가 없는 경우
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()  # 키워드 지우기

driver.quit()  # 크롤링 종료

In [ ]:
# 현재 폴더에 csv파일로 저장
ral_extra_res.to_csv('./랠릿_다트_추가.csv')

### 캐치 크롤링
- 크롤링할 컬럼: 기업명, 기업형태, 매출액, 사원수, 업종분류, 설립연도
- 절차
1. url로 기업명 입력
2. 기업정보 사이트 접속
3. 내용 크롤링
4. concat
5. 다음 url 입력

위 절차를 반복합니다.

#### 프로그래머스

In [ ]:
# 빈 프레임 생성
pro_extra2_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in pro_corp_name:  # 기업명 하나씩 불러오기
    try:
        url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'

        driver= webdriver.Chrome()
        driver.get(url)

        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()  # 기업정보 클릭
        time.sleep(0.2)

        # 기업명
        corp_name = name
        # 기업형태
        corp_type = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[2]/td[1]').text
        # 매출액
        sales_money = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[1]').text
        # 사원수
        employee = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[2]').text
        # 업종분류
        sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[4]/td[1]').text
        # 설립연도
        corp_year = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[1]/td[2]').text

        pro_extra2_temp = pd.DataFrame([corp_name,corp_type,sales_money,employee,sector,corp_year], index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        pro_extra2_res = pd.concat([pro_extra2_res,pro_extra2_temp])

        time.sleep(0.2)
    except:  # 기업정보가 없어서 클릭되지 않는 경우
        pass
    
    driver.quit()  # 크롤링 종료

In [ ]:
# 현재 폴더에 csv파일로 저장
pro_extra2_res.to_csv('./프로그래머스_캐치_추가.csv')

#### 랠릿

In [ ]:
# 빈 프레임 생성
ral_extra2_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in ral_corp_name:  # 기업명 하나씩 불러오기
    try:
        url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'

        driver= webdriver.Chrome()
        driver.get(url)

        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()  # 기업정보 클릭
        time.sleep(0.2)

        # 기업명
        corp_name = name
        # 기업형태
        corp_type = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[2]/td[1]').text
        # 매출액
        sales_money = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[1]').text
        # 사원수
        employee = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[2]').text
        # 업종분류
        sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[4]/td[1]').text
        # 설립연도
        corp_year = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[1]/td[2]').text

        ral_extra2_temp = pd.DataFrame([corp_name,corp_type,sales_money,employee,sector,corp_year], index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        ral_extra2_res = pd.concat([ral_extra2_res,ral_extra2_temp])

        time.sleep(0.2)
    except:  # 기업정보가 없어서 클릭되지 않는 경우
        pass
    
    driver.quit()  # 크롤링 종료

In [ ]:
# 현재 폴더에 csv파일로 저장
ral_extra2_res.to_csv('./랠릿_캐치_추가.csv')